In [1]:
import os
import json
from pprint import pprint as pp
import pandas as pd
from datetime import datetime
import time
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import sklearn
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.metrics import silhouette_samples, silhouette_score
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE


%matplotlib inline

In [51]:
bool_plot = False
save_pickle = True

In [5]:
df = pd.read_pickle('tag_prop.pkl')
tag_mat = df.as_matrix(columns=None)

In [6]:
use_adjust = False
if use_adjust:
    df_adjust = df
    top_n = 12
    for column in df:
        df_adjust[column] =  (df[column] - min(df[column].nlargest(top_n))).apply(np.ceil).apply(int)
    df_adjust = df_adjust.transpose()

In [7]:

df = df.transpose()

In [8]:
tag_names = df.columns.values 

In [9]:
if bool_plot:
    X_reduced = TruncatedSVD(n_components=50, random_state=0).fit_transform(df)
    X_embedded = TSNE(n_components=2, perplexity=40, verbose=2).fit_transform(X_reduced)

In [50]:
cluster_n = 300
k_means = KMeans(n_clusters=cluster_n, init='k-means++', max_iter=100, n_init=10,)
k_means.fit(df)

KMeans(copy_x=True, init='k-means++', max_iter=100, n_clusters=300, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [10]:
'''determine appropriate number of clusters'''

# X_nfit = df_adjust.as_matrix()
# #test_clusters = [20,30,40,50,60,70,80,90,100]
# #test_clusters = [30,50,70,100]
# #test_clusters = [20,40,80,100,200,300,400,500]
# test_clusters = [200,400]
# sil_a_list = list()
# for nc in test_clusters:
#     k_means = KMeans(n_clusters=nc, init='k-means++', max_iter=100, n_init=1,)
#     k_means.fit(X_nfit)
#     cl_x = k_means.fit_predict(X_nfit)
#     silhouette_avg = silhouette_score(X_nfit, cl_x)
#     sil_a_list.append(silhouette_avg)
#     print("For n_clusters =", nc,
#       "The average silhouette_score is :", silhouette_avg)

    #sample_silhouette_values = silhouette_samples(X_nfit, cl_x)

'determine appropriate number of clusters'

In [11]:
#plt.plot(test_clusters,sil_a_list)

In [53]:
k_means.labels_
cluster_bool = list()
for i in range(cluster_n):
    cur_cluster = k_means.labels_ == i
    cluster_bool.append(cur_cluster)

In [54]:
appids = df.index

In [55]:
df_labels = pd.DataFrame({'appid':appids,'cluster':k_means.labels_})
if save_pickle:
    df_labels.to_pickle('df_kmeans.pkl')

In [15]:
if bool_plot:
    '''red to green color map'''
    cdict = {'red':   ((0.0, 0.0, 0.0),
                       (0.5, 0.0, 0.0),
                       (1.0, 1.0, 1.0)),
             'blue':  ((0.0, 0.0, 0.0),
                       (1.0, 0.0, 0.0)),
             'green': ((0.0, 0.0, 1.0),
                       (0.5, 0.0, 0.0),
                       (1.0, 0.0, 0.0))}

    cmap = mcolors.LinearSegmentedColormap(
    'my_colormap', cdict, 100)
    base_val = 100

In [16]:
if bool_plot:
    qmaps = ['Accent', 'Dark2', 'Paired', 'Pastel1','Pastel2', 'Set1', 'Set2', 'Set3', 'Spectral']
    cmap_name = qmaps[8]
    cmap=plt.get_cmap(cmap_name)
    base_val = 255
    
    fig,ax = plt.subplots(1,1,figsize=(10, 10))
    for i in range(cluster_n):
        cmap_val = int(( (i * 1.0) / (cluster_n - 1)) * base_val)
        ax.plot(X_embedded[cluster_bool[i], 0], X_embedded[cluster_bool[i], 1], marker = '.', ls = '', color = cmap(cmap_val))




In [17]:
order_centroids = k_means.cluster_centers_.argsort()[:, ::-1]

In [18]:
order_centroids.shape

(400, 311)

In [39]:
'''print out the top labels in each category'''
top_n_labels = 4
order_centroids.shape
for i in range(order_centroids.shape[0]):
    tn = tag_names[order_centroids[i,0:top_n_labels]]
    tn_out = [x.encode('ascii')for x in tn]
    print tn_out

NameError: name 'order_centroids' is not defined

In [46]:
df_labels.head()

,appid,cluster,game_name
0,10,268,Counter-Strike
1,20,268,Team Fortress Classic
2,30,166,Day of Defeat
3,40,268,Deathmatch Classic
4,50,21,Half-Life: Opposing Force


In [56]:
df_other = pd.read_pickle('steam_features.pkl')
df_gn = df_other[['game_name','appid']]

In [57]:
df_labels = pd.merge(df_labels, df_gn, left_on='appid', right_on='appid', how='inner')

In [58]:
df_labels.sort('cluster')

,appid,cluster,game_name
2782,310380,0,Fractured Space
4174,377150,0,Tactical Genius
1342,222900,0,Dead Island: Epidemic
2629,301520,0,Robocraft
2159,277950,0,Deadbreed®
1722,251970,0,Sins of a Dark Age
4188,378280,0,Pump-Action Captain
3022,320590,0,Warside
2296,285580,0,ACE - Arena: Cyber Evolution
1049,200110,0,Nosgoth


In [60]:
df_pc = pd.read_pickle('df_price_cluster.pkl')

In [62]:
df_pc_out = pd.merge(df_pc[['appid','median_initial_price']],df_labels,left_on='appid', right_on='appid')

In [63]:
if save_pickle:
    df_pc.to_pickle('df_price_cluster.pkl')